# Homework 2

## Bayes Spam Filter

First, I make a list of all mail documents.

In [139]:
# mail corpus                                                                                                                                                                                
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]                                                                                        
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]       

all_words = []

def all_counts(corpus, list):
    for mail in corpus:
        for word in mail:
            if word.lower() not in all_words:
                all_words.append(word.lower())
            list.append(word.lower())


Then make lists of spam and ham words and get the lengths of each

In [140]:
# mail docs
spam = []
ham = []
all_counts(spam_corpus, spam)
all_counts(ham_corpus, ham)    

print(all_words)
print(spam)


['i', 'am', 'spam', 'do', 'not', 'like', 'that', 'spamiam', 'green', 'eggs', 'and', 'ham']
['i', 'am', 'spam', 'spam', 'i', 'am', 'i', 'do', 'not', 'like', 'that', 'spamiam']


Using the list of all documents and the spam and ham lists, I created a hashtable of words and their frequencies in both the spam corpus and the ham corpus.

In [141]:
# count each word and its frequency for each spam and ham
def counter(list):
    dict = {}
    for word in all_words:
        if list.count(word) == 0:
            dict[word] = 1
        else:
            dict[word] = list.count(word)
    return dict

# hashtable of good and bad words and their frequencies
good_words_freqs = counter(ham)
bad_words_freqs = counter(spam)



n_good = sum(good_words_freqs.values())
n_bad = sum(bad_words_freqs.values())

This function implements Paul Graham's probability calculation algorithm

In [142]:
def probability_calc(word):
    '''
    Calculate spam probability for each token
    '''
    g = 2 * good_words_freqs[word] # double the count of ham words to reduce false positives
    b = bad_words_freqs[word]
    if g + b > 1:
        return max(0.01, min(0.99, min(1.0, b/n_bad) / min(1.0, g/n_good) + min(1.0, b/n_bad)))
    
    return 0

Using this algorithm, probabilities are assigned to each word in both corpora. If a word occurs only in the ham corpus, it is assigned a probability of 0.01. If it is found only in the spam corpus, it gets a probability of 0.9. If found in both, Graham's algorithm is used to find a probability. If it is not in either corpus, it is assigned a probability of 0.4.

In [143]:
# Probability of spam hashtable for each word 
all_word_probs = {}

for word in all_words:
        p = probability_calc(word)
        if word not in ham and word not in spam:
            all_word_probs[word] = 0.4
        elif word not in spam and word in ham:
            all_word_probs[word] = 0.01
        elif word in spam and word not in ham:
            all_word_probs[word] = 0.9
        elif word in spam and word in ham:
            all_word_probs[word] = p

print(all_word_probs)

{'i': 0.84375, 'am': 0.9, 'spam': 0.9, 'do': 0.28125, 'not': 0.9, 'like': 0.5, 'that': 0.9, 'spamiam': 0.9, 'green': 0.01, 'eggs': 0.01, 'and': 0.01, 'ham': 0.01}


This scoring function finds the probability that a document is spam by finding the product of the probabilities of its words.

In [146]:
def score(mail, probabilities):
    prob = 1
    for word in mail:
        print("\t", word, ":\t ", all_word_probs[word.lower()])
        prob*=all_word_probs[word.lower()]
    print(prob)
    return prob



In [147]:
print("Ham: ")
[print(mail, " ", score(mail, all_word_probs)) for mail in ham_corpus]

print()
print("Spam: ")
[print(mail, " ", score(mail, all_word_probs)) for mail in spam_corpus]

Ham: 
	 do :	  0.28125
	 i :	  0.84375
	 like :	  0.5
	 green :	  0.01
	 eggs :	  0.01
	 and :	  0.01
	 ham :	  0.01
1.1865234375000002e-09
['do', 'i', 'like', 'green', 'eggs', 'and', 'ham']   1.1865234375000002e-09
	 i :	  0.84375
	 do :	  0.28125
0.2373046875
['i', 'do']   0.2373046875

Spam: 
	 I :	  0.84375
	 am :	  0.9
	 spam :	  0.9
	 spam :	  0.9
	 I :	  0.84375
	 am :	  0.9
0.46708681640625005
['I', 'am', 'spam', 'spam', 'I', 'am']   0.46708681640625005
	 I :	  0.84375
	 do :	  0.28125
	 not :	  0.9
	 like :	  0.5
	 that :	  0.9
	 spamiam :	  0.9
0.08649755859375001
['I', 'do', 'not', 'like', 'that', 'spamiam']   0.08649755859375001


[None, None]

This is called Baysian filtering because it uses Bayes' Rule to calculate the probability that a given word is spam.

## 2

In [7]:
import sys                                                                                                      
sys.path.append('/home/james/Documents/Calvin/CS-344/cs344-code/tools/aima')                                    
from probability import BayesNet, enumeration_ask, elimination_ask, gibbs_ask, rejection_sampling, likelihood_weighting                                                                      


In [27]:
# Utility variables                                                                                                                                                                          
T, F = True, False                                                                                                                                                                           

wet_lawns = BayesNet([                                                                                                                                                                       
  ('Cloudy', '', 0.5),                                                                                                                                                                     
  ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.5}),                                                                                                                                               
  ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),                                                                                                                                                    
  ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.90, (F, T): 0.9, (F, F): 0.0})                                                                                                   
  ])                                                                                                                                                                                       


### 2. b.                                                                                                                       
A full join probability distribution would have 2*2*4 = 16 values                                                                                                                            
### 2. c.                                                                                                                                                                                    
The Bayes Network for this domain has 9 independent values with the conditional independence relations   

In [115]:

#d.                                                                                                                                                                                         
print('P(Cloudy)')                                                                                                                                                                           

# P(Cloudy) = 0.5 from the network <0.5, 0.5>
print(enumeration_ask('Cloudy', dict(),  wet_lawns).show_approx())                                                                                                                           


print('P(Sprinker | cloudy')                                                                                                                                                                 

# P(Sprinkler | Cloudy) = 0.1, <0.1, 0.9>
print(enumeration_ask('Sprinkler', dict(Cloudy=T), wet_lawns).show_approx())                                                                                                                  

print('P(Cloudy| the sprinkler is running and it’s not raining)')                                                                                                                            

# a<0.5*0.1*0.2, 0.5*0.5*0.8>
# = 4.76<0.01, 0.2>
# = <0.476, 0.952>
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), wet_lawns).show_approx())                                                                                                          

print('P(WetGrass | it’s cloudy, the sprinkler is running and it’s raining)')                                                                                                                

# <0.99, 0.01> from Bayes Net
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), wet_lawns).show_approx())                                                                                              

print('P(Cloudy | the grass is not wet)')                                                                                                                                                    

# 
print(enumeration_ask('Cloudy', dict(WetGrass=F), wet_lawns).show_approx()) 

P(Cloudy)


NameError: name 'enumeration_ask' is not defined